In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

Using TensorFlow backend.


In [2]:
data = pd.read_csv("final_dataset.csv", encoding="latin1")

data = data.fillna(method="ffill")
data.tail(5)

,given_name_A,surname_A,street_number_A,address_1_A,address_2_A,suburb_A,postcode_A,state_A,date_of_birth_A,soc_sec_id_A,...,surname_B,street_number_B,address_1_B,address_2_B,suburb_B,postcode_B,state_B,date_of_birth_B,soc_sec_id_B,match
999,jade,browne,407.0,sabine close,brentwood vlge,leichhardt,4700,vic,19810923.0,8107465,...,browne,407.0,sabine close,brentwoidvlge,leichhardt,4708,vic,19810923.0,8107465,1
1000,taalia,clarke,4.0,notting court,aboriginal nhulunbuy hostel,lindfield,2196,qld,19260303.0,8505822,...,clarke,4.0,notting court,aboriginal nhulunbuy hostel,lindfield,2195,qld,19260303.0,8505822,1
1001,taalia,dent,4.0,chillagoe street,myliton,hamilton,7325,nsw,19270825.0,9350791,...,clarkd,4.0,streeter place,riverside plaza,port lincoln,2090,sa,19700607.0,1250298,0
1002,madison,nguyen,101.0,brisbane avenue,billsblock,richardson,3658,nsw,19450606.0,4699715,...,shepherd,18.0,ballumbir street,blue hills,corrmal,3168,wa,19170819.0,8120482,0
1003,connor,paine,10.0,adair street,kangarilla,warren,3194,sa,19140306.0,5213108,...,paine,10.0,adair strret,blue hills,warre n,3194,wa,19140306.0,5213108,1


In [3]:
data.dtypes

given_name_A        object
surname_A           object
street_number_A    float64
address_1_A         object
address_2_A         object
suburb_A            object
postcode_A           int64
state_A             object
date_of_birth_A    float64
soc_sec_id_A         int64
given_name_B        object
surname_B           object
street_number_B    float64
address_1_B         object
address_2_B         object
suburb_B            object
postcode_B           int64
state_B             object
date_of_birth_B    float64
soc_sec_id_B         int64
match                int64
dtype: object

In [4]:
def case_conv(s) :
    k = str(s).lower()

    return k


wordwa = []
dd = list(data.columns)
#dd = dd[:,-1]
#print(dd)

data["street_number_A"] = data["street_number_A"].apply(case_conv)
data["street_number_B"] = data["street_number_B"].apply(case_conv)
data["date_of_birth_A"] = data["date_of_birth_A"].apply(case_conv)
data["date_of_birth_B"] = data["date_of_birth_B"].apply(case_conv)

for i in range(len(dd)-1) :
    #dd[i] = dd[i].apply(case_conv)
    w = list(set(data[dd[i]].values))
    for w1 in w :
        wordwa.append(w1)
        
#wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
    
print("Words[0:10] : ")
print(words[0:10])
n_words = len(words); 
n_words

chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print("Number of characters : " + str(n_chars))

char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

Words[0:10] : 
['nikita', 'ella', 'erin', 'thomas', 'lachlan', 'bianca', 'lewis', 'ellie', 'tansy', 'zachary']
Number of characters : 43


In [5]:
print(chars)

{'x', '.', '/', 'i', 'j', 'q', '3', '2', 'u', 'd', 'k', 'm', 's', '4', '6', '8', "'", 'r', 'f', '-', 'z', '9', '&', '5', 'n', '7', 'a', '0', 'b', 'l', 'v', 'h', 't', 'c', 'p', ' ', '(', 'e', 'w', 'g', 'o', '1', 'y'}


In [6]:
max_word_length = 25
embedded_char_vector_length = 45 # Length of character dictionary
char_feature_output = 10

X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [7]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(embedded_char_vector_length, 16, input_length = max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 16)            720       
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10)                880       
Total params: 1,600
Trainable params: 1,600
Non-trainable params: 0
_________________________________________________________________


In [8]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    pred = pred.lower()
    #print(pred)
    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        #print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [9]:
def pred_new_vec(l) :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [10]:
def form_list(R) :
    lR = []
    for r in R :
        a = pred_new_num_enc(r)
        #print(a)
        #print("\n")
        a = pred_new_vec(a)
        #print(a)
        #print("\n")    
        lR.append(a)
        #print(lR)
        
    lR = np.array(lR)
    #print(lR.shape)

    #np.reshape(lR1,(5,10))
    lR = lR[:,0,:]

    #print(lR1)
    #print("\n")
    
    #print(lR.shape)
    #print("\n")
    #print(lR)
    #print(lR.shape)
    lR = lR.flatten()

    #print(lR.shape)
    
    lR = list(lR)
    
    #print(lR)
    #print(len(lR))
    
    return lR

In [11]:
dataset = pd.read_csv("final_dataset.csv")

In [12]:
dataset = dataset.fillna('xxxxxx')

In [13]:
dataset = dataset.replace(np.nan,'xxxxxx')

In [14]:
'''dataset['given_name_A'] = dataset['given_name_A'].fillna('xxxxxx')
dataset['given_name_B'] = dataset['given_name_B'].fillna('xxxxxx')

dataset['surname_A'] = dataset['surname_A'].fillna('xxxxxx')
dataset['surname_B'] = dataset['surname_B'].fillna('xxxxxx')

dataset['address_1_A'] = dataset['address_1_A'].fillna('xxxxxx')
dataset['address_1_B'] = dataset['address_1_B'].fillna('xxxxxx')

dataset['address_2_A'] = dataset['address_2_A'].fillna('xxxxxx')
dataset['address_2_B'] = dataset['address_2_B'].fillna('xxxxxx')

dataset['suburb_A'] = dataset['suburb_A'].fillna('xxxxxx')
dataset['suburb_B'] = dataset['suburb_B'].fillna('xxxxxx')

dataset['state_A'] = dataset['state_A'].fillna('xxxxxx')
dataset['state_B'] = dataset['state_B'].fillna('xxxxxx')'''

"dataset['given_name_A'] = dataset['given_name_A'].fillna('xxxxxx')\ndataset['given_name_B'] = dataset['given_name_B'].fillna('xxxxxx')\n\ndataset['surname_A'] = dataset['surname_A'].fillna('xxxxxx')\ndataset['surname_B'] = dataset['surname_B'].fillna('xxxxxx')\n\ndataset['address_1_A'] = dataset['address_1_A'].fillna('xxxxxx')\ndataset['address_1_B'] = dataset['address_1_B'].fillna('xxxxxx')\n\ndataset['address_2_A'] = dataset['address_2_A'].fillna('xxxxxx')\ndataset['address_2_B'] = dataset['address_2_B'].fillna('xxxxxx')\n\ndataset['suburb_A'] = dataset['suburb_A'].fillna('xxxxxx')\ndataset['suburb_B'] = dataset['suburb_B'].fillna('xxxxxx')\n\ndataset['state_A'] = dataset['state_A'].fillna('xxxxxx')\ndataset['state_B'] = dataset['state_B'].fillna('xxxxxx')"

In [15]:
c = list(dataset.columns)
type(c)

list

In [16]:
null_value_stats = dataset.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

In [17]:
dataset.dtypes

given_name_A       object
surname_A          object
street_number_A    object
address_1_A        object
address_2_A        object
suburb_A           object
postcode_A          int64
state_A            object
date_of_birth_A    object
soc_sec_id_A        int64
given_name_B       object
surname_B          object
street_number_B    object
address_1_B        object
address_2_B        object
suburb_B           object
postcode_B          int64
state_B            object
date_of_birth_B    object
soc_sec_id_B        int64
match               int64
dtype: object

In [18]:
def dob_conv(dob) :
    k=str(dob)

    return k

In [19]:
dataset["street_number_A"] = dataset["street_number_A"].apply(dob_conv)
dataset["street_number_B"] = dataset["street_number_B"].apply(dob_conv)

dataset["date_of_birth_A"] = dataset["date_of_birth_A"].apply(dob_conv)
dataset["date_of_birth_B"] = dataset["date_of_birth_B"].apply(dob_conv)

dataset["postcode_A"] = dataset["postcode_A"].apply(dob_conv)
dataset["postcode_B"] = dataset["postcode_B"].apply(dob_conv)

dataset["soc_sec_id_A"] = dataset["soc_sec_id_A"].apply(dob_conv)
dataset["soc_sec_id_B"] = dataset["soc_sec_id_B"].apply(dob_conv)


for i,r in dataset.iterrows() :
    r["date_of_birth_A"] = r["date_of_birth_A"].strip('.0')
    r["date_of_birth_B"] = r["date_of_birth_B"].strip('.0')

In [20]:
dataset.dtypes

given_name_A       object
surname_A          object
street_number_A    object
address_1_A        object
address_2_A        object
suburb_A           object
postcode_A         object
state_A            object
date_of_birth_A    object
soc_sec_id_A       object
given_name_B       object
surname_B          object
street_number_B    object
address_1_B        object
address_2_B        object
suburb_B           object
postcode_B         object
state_B            object
date_of_birth_B    object
soc_sec_id_B       object
match               int64
dtype: object

In [21]:
R_A = c[:10]
R_B = c[10:20]

In [22]:
print(R_A)
print(R_B)
print(c[-1])

['given_name_A', 'surname_A', 'street_number_A', 'address_1_A', 'address_2_A', 'suburb_A', 'postcode_A', 'state_A', 'date_of_birth_A', 'soc_sec_id_A']
['given_name_B', 'surname_B', 'street_number_B', 'address_1_B', 'address_2_B', 'suburb_B', 'postcode_B', 'state_B', 'date_of_birth_B', 'soc_sec_id_B']
match


In [23]:
'''x1 = []
x2 = []

d1 = dataset.iloc[0,:]
d1["date_of_birth_A"] = d1["date_of_birth_A"].replace('.0','')
d1["date_of_birth_B"] = d1["date_of_birth_B"].replace('.0','')

for j in R_A :
    x1.append(d1[j])
for jj in R_B :
    x2.append(d1[jj])
    
lr5 = form_list(x1)
lr6 = form_list(x2)'''

'x1 = []\nx2 = []\n\nd1 = dataset.iloc[0,:]\nd1["date_of_birth_A"] = d1["date_of_birth_A"].replace(\'.0\',\'\')\nd1["date_of_birth_B"] = d1["date_of_birth_B"].replace(\'.0\',\'\')\n\nfor j in R_A :\n    x1.append(d1[j])\nfor jj in R_B :\n    x2.append(d1[jj])\n    \nlr5 = form_list(x1)\nlr6 = form_list(x2)'

In [24]:
mlp_data = []
mlp_labels = []

for i,row in dataset.iterrows() :
    #print(row)
    rec = []
    r3 = []
    r4 = []
    for j in R_A :
        r3.append(row[j])
    for jj in R_B :
        r4.append(row[jj])
        
    lr3 = form_list(r3)
    lr4 = form_list(r4)
    
    rec = lr3 + lr4
    mlp_data.append(rec)
    mlp_labels.append(row['match'])

In [25]:
np.array(mlp_data).shape

(1004, 200)

In [26]:
np.array(mlp_labels).shape

(1004,)

In [29]:
mlp_data[0:2]

[[-0.030904831,
  0.00871507,
  -0.0047627552,
  0.011497563,
  0.051942263,
  -0.0064711277,
  -0.017224953,
  0.0042134398,
  0.002749718,
  -0.0078077386,
  -0.030980444,
  0.008943697,
  -0.0048209126,
  0.011522563,
  0.05199436,
  0.0053345496,
  -0.011179793,
  -0.0032401143,
  -0.006778661,
  -0.009482675,
  -0.030956386,
  0.00895835,
  -0.0048448914,
  0.01150308,
  0.051989757,
  0.011733261,
  -0.012727662,
  0.022142608,
  -0.053024564,
  -0.021142012,
  -0.030074654,
  0.008513327,
  -0.005750151,
  0.01114364,
  0.051005375,
  0.011555677,
  0.0041836966,
  -0.023212422,
  0.00062809687,
  0.00988952,
  -0.02201425,
  0.009101105,
  -0.00962364,
  0.009124744,
  0.035173226,
  -0.018074598,
  -0.0024586858,
  -0.006227941,
  0.022597594,
  0.011800706,
  -0.030659307,
  0.008652126,
  -0.0051707933,
  0.011461246,
  0.051607966,
  -0.0015775851,
  0.012117048,
  -0.0041130725,
  0.023701118,
  -0.002393057,
  -0.030943999,
  0.00886784,
  -0.0048050364,
  0.011495987,
  

In [28]:
mlp_labels[0]

1

In [30]:
mlp = pd.DataFrame(mlp_data)

In [31]:
mlp['labels'] = mlp_labels

In [32]:
mlp.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,labels
0,-0.030905,0.008715,-0.004763,0.011498,0.051942,-0.006471,-0.017225,0.004213,0.002750,-0.007808,...,0.008838,-0.004889,0.011431,0.051930,-0.006360,-0.011561,-0.008498,0.003132,0.015139,1
1,-0.030966,0.008694,-0.004702,0.011530,0.051964,-0.020023,-0.010591,0.019676,0.022855,-0.014863,...,0.008727,-0.004774,0.011487,0.051940,-0.010006,0.000759,0.009258,0.006597,-0.003244,1
2,-0.031000,0.008893,-0.004778,0.011534,0.051994,-0.001351,0.003744,0.010609,0.000955,0.007380,...,0.008888,-0.004808,0.011522,0.051980,0.000363,0.001830,0.001659,0.003050,0.026492,0
3,-0.030956,0.008958,-0.004845,0.011503,0.051990,0.011733,-0.012728,0.022143,-0.053025,-0.021142,...,0.008842,-0.004858,0.011453,0.051945,-0.004986,-0.002946,-0.005060,0.013522,0.014156,0
4,-0.030923,0.008799,-0.004795,0.011508,0.051957,0.000581,0.006004,0.006903,-0.004044,-0.004975,...,0.008887,-0.004809,0.011526,0.051977,-0.003427,-0.009142,0.001329,0.000029,-0.000493,0


In [33]:
mlp.to_csv('nn_data.csv' , index = False)

In [35]:
nd = pd.read_csv("nn_data.csv")
nd.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,labels
0,-0.030905,0.008715,-0.004763,0.011498,0.051942,-0.006471,-0.017225,0.004213,0.002750,-0.007808,...,0.008838,-0.004889,0.011431,0.051930,-0.006360,-0.011561,-0.008498,0.003132,0.015139,1
1,-0.030966,0.008694,-0.004702,0.011530,0.051964,-0.020023,-0.010591,0.019676,0.022855,-0.014863,...,0.008727,-0.004774,0.011487,0.051940,-0.010006,0.000759,0.009258,0.006597,-0.003244,1
2,-0.031000,0.008893,-0.004778,0.011534,0.051994,-0.001351,0.003744,0.010609,0.000955,0.007380,...,0.008888,-0.004808,0.011522,0.051980,0.000363,0.001830,0.001659,0.003050,0.026492,0
3,-0.030956,0.008958,-0.004845,0.011503,0.051990,0.011733,-0.012728,0.022143,-0.053025,-0.021142,...,0.008842,-0.004858,0.011453,0.051945,-0.004986,-0.002946,-0.005060,0.013522,0.014156,0
4,-0.030923,0.008799,-0.004795,0.011508,0.051957,0.000581,0.006004,0.006903,-0.004044,-0.004975,...,0.008887,-0.004809,0.011526,0.051977,-0.003427,-0.009142,0.001329,0.000029,-0.000493,0


In [37]:
X = nd.iloc[:,:-1]
y = nd.iloc[:,-1]

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [42]:
print(X_train.shape,X_test.shape, Y_train.shape, Y_test.shape)

(803, 200) (201, 200) (803,) (201,)


In [47]:
# create model

mlp_model = Sequential()
mlp_model.add(Dense(100, input_dim=200, activation='relu'))
mlp_model.add(Dense(60, activation='relu'))
mlp_model.add(Dense(1, activation='sigmoid'))

In [48]:
# Compile model
mlp_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
mlp_model.fit(X_train, Y_train, epochs=150, batch_size=50)

Epoch 1/150
803/803 [==============================] - 1s 1ms/step - loss: 0.6604 - acc: 0.6887
Epoch 2/150
803/803 [==============================] - 0s 83us/step - loss: 0.6112 - acc: 0.6974
Epoch 3/150
803/803 [==============================] - 0s 65us/step - loss: 0.6110 - acc: 0.6974
Epoch 4/150
803/803 [==============================] - 0s 59us/step - loss: 0.6071 - acc: 0.6974
Epoch 5/150
803/803 [==============================] - 0s 57us/step - loss: 0.6047 - acc: 0.6974
Epoch 6/150
803/803 [==============================] - 0s 59us/step - loss: 0.6010 - acc: 0.6974
Epoch 7/150
803/803 [==============================] - 0s 57us/step - loss: 0.5966 - acc: 0.6974
Epoch 8/150
803/803 [==============================] - 0s 60us/step - loss: 0.5934 - acc: 0.6974
Epoch 9/150
803/803 [==============================] - 0s 57us/step - loss: 0.5871 - acc: 0.6974
Epoch 10/150
803/803 [==============================] - 0s 59us/step - loss: 0.5812 - acc: 0.6974
Epoch 11/150
803/803 [========

803/803 [==============================] - 0s 52us/step - loss: 0.3330 - acc: 0.8705
Epoch 85/150
803/803 [==============================] - 0s 49us/step - loss: 0.3264 - acc: 0.8792
Epoch 86/150
803/803 [==============================] - 0s 56us/step - loss: 0.3137 - acc: 0.8780
Epoch 87/150
803/803 [==============================] - 0s 51us/step - loss: 0.3098 - acc: 0.8879
Epoch 88/150
803/803 [==============================] - 0s 54us/step - loss: 0.3109 - acc: 0.8867
Epoch 89/150
803/803 [==============================] - 0s 71us/step - loss: 0.2979 - acc: 0.8917
Epoch 90/150
803/803 [==============================] - 0s 73us/step - loss: 0.2892 - acc: 0.8929
Epoch 91/150
803/803 [==============================] - 0s 86us/step - loss: 0.2919 - acc: 0.9066
Epoch 92/150
803/803 [==============================] - 0s 77us/step - loss: 0.2843 - acc: 0.8991
Epoch 93/150
803/803 [==============================] - 0s 57us/step - loss: 0.2710 - acc: 0.9141
Epoch 94/150
803/803 [===========

In [50]:
# evaluate the model
scores = mlp_model.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (mlp_model.metrics_names[1], scores[1]*100))

803/803 [==============================] - 0s 237us/step

acc: 98.38%


In [54]:
from sklearn.metrics import confusion_matrix,precision_score

predictions = mlp_model.predict(X_test)
# round predictions
rounded = [round(x[0]) for x in predictions]

In [55]:
y_pred1 = np.array(rounded , dtype = 'int64')

In [56]:
confusion_matrix(Y_test,y_pred1)

array([[133,   7],
       [ 16,  45]], dtype=int64)

In [57]:
precision_score(Y_test,y_pred1)

0.8653846153846154

In [58]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test,y_pred1)

0.8855721393034826